# Assignment V2

## Imports

In [1]:
#checks for environment
import sys
print(sys.executable)

c:\Users\yonab\assignment-V2\.venv\Scripts\python.exe


In [2]:
import pandas as pd
import importlib

In [ ]:
#import all modules including necessary libraries
from lib.openai_client import*
from lib.paths import DATA_DIR,PROJECT_ROOT,CSV_DIR, TXT_DIR, PRODUCTS_JSON_PATH, PRODUCTS_CSV_PATH
importlib.reload(lib.data_loading)
from lib.data_loading import load_csv_products, load_txt_products
import lib.constants
importlib.reload(lib.constants)
from lib.text_extraction import extract_products_from_txt_df
from lib.parsing import normalize_csv_products
from lib.validation import add_validation_columns
from lib.catalog import build_catalog_for_llm

import lib.chatbot
import lib.ui_catalog_chat

importlib.reload(lib.chatbot)
importlib.reload(lib.ui_catalog_chat)

from lib.chatbot import create_catalog_system_context
from lib.ui_catalog_chat import build_catalog_chat_dashboard
from lib.export import export_products


## Data loading


Récuperer la donnée des fichiers txt, puis csv et la display

In [4]:
df_csv_raw = load_csv_products(CSV_DIR)
df_txt_raw = load_txt_products(TXT_DIR)


In [6]:
df_csv_raw

,product_name,description,price,colors,category,stock,features,raw_file
0,Bureau assis-debout FlexDesk,Bureau réglable en hauteur pour alterner entre...,399.0,blanc; chêne clair,office,En stock,Réglage électrique; Mémoire 3 positions; Plate...,catalogue_multi.csv
1,Lampe de bureau Lumina Pro,Lampe LED avec luminosité et température de co...,59.9,noir; argent,"""electronics / lighting""",En stock,Bras articulé; Lumière chaude/froide; Port USB...,catalogue_multi.csv
2,Écran 27'' UltraView,"Écran 27 pouces avec bordures fines, idéal pou...",279.0,noir,electronics,Stock limité,Résolution 1440p; Mode low-blue light; Pied ré...,catalogue_multi.csv
3,Chaise de gaming ComfortSeat,Chaise confortable avec support lombaire pour ...,199.0,noir / rouge,furniture,En stock,Dossier inclinable; Coussin lombaire; Roulette...,catalogue_multi.csv
4,Sac à dos UrbanTech,Sac à dos pour laptop avec multiples compartim...,89.0,noir; bleu marine,accessory,En stock,Compartiment 15 pouces; Poche cachée; Port USB...,catalogue_multi.csv
5,Casque sans fil FocusLite,Casque léger sans fil pensé pour le travail co...,79.9,gris clair,electronics / audio,En stock,Réduction passive de bruit; Micro intégré; Con...,catalogue_multi.csv
6,Tapis de souris XL Work&Play,"Grand tapis de souris pour clavier + souris, i...",29.9,noir,accessory,En stock,Base antidérapante; Surface lisse; Bords renfo...,catalogue_multi.csv
7,Montre connectée MoveFit,Montre connectée pour le suivi d’activité et l...,149.0,rose; noir,wearable,En stock,GPS intégré; Suivi du sommeil; Étanche 5ATM,catalogue_multi.csv
8,Chaise de bureau ErgoFlex,Chaise ergonomique avec soutien lombaire régla...,249.0,noir,furniture,Stock limité,Accoudoirs 3D; Dossier mesh respirant; Hauteur...,chaise_bureau.csv
9,Clavier mécanique FusionKey 75,"Clavier mécanique compact 75 %, switches tacti...",119.9,"noir, blanc cassé",electronics,En stock,Switches hot-swap; Rétroéclairage ajustable; C...,clavier_mecanique.csv


In [7]:
df_txt_raw.head()

,raw_file,raw_text
0,fiche1_montre.txt,Produit : Montre connectée AX490\nDescription ...
1,fiche2_casque.txt,Nom du produit : Casque audio NovaSound Pro\nC...
2,fiche3_sac_a_dos.txt,Produit : Sac à dos UrbanTrail 25L\nCe sac à d...
3,fiche4_tee_shirt.txt,Produit : T-shirt technique RunDry\nDescriptio...
4,fiche5_lampe.txt,Nom : Lampe de bureau FlexiBright\nUne lampe L...


In [9]:
df_txt_raw

,raw_file,raw_text
0,fiche1_montre.txt,Produit : Montre connectée AX490\nDescription ...
1,fiche2_casque.txt,Nom du produit : Casque audio NovaSound Pro\nC...
2,fiche3_sac_a_dos.txt,Produit : Sac à dos UrbanTrail 25L\nCe sac à d...
3,fiche4_tee_shirt.txt,Produit : T-shirt technique RunDry\nDescriptio...
4,fiche5_lampe.txt,Nom : Lampe de bureau FlexiBright\nUne lampe L...
5,fiches_multi.txt#part1,Produit : Montre connectée WorkLife X2\nDescri...
6,fiches_multi.txt#part2,Nom du produit : Lampe de bureau MorningBright...
7,fiches_multi.txt#part3,Produit : Support d’ordinateur ErgoLift\nDescr...
8,fiches_multi.txt#part4,Nom du produit : Casque filaire OfficeTalk\nCa...


In [10]:
print(df_txt_raw.loc[0, "raw_text"])

Produit : Montre connectée AX490
Description : Une montre élégante pensée pour le quotidien, avec capteur cardiaque, GPS intégré et une autonomie d'environ 4 jours en usage normal.
Prix : 129 euros TTC
Couleurs disponibles : noir / bleu
Catégorie : wearable
Stock : En rupture pour l’instant
Points forts :
 - Ultra légère
 - Compatible iOS et Android
 - Notifications appel / SMS / apps



In [11]:
print(df_txt_raw.loc[7, "raw_text"])

Produit : Support d’ordinateur ErgoLift
Description : Support surélevé pour laptop, pour améliorer la posture et libérer de l’espace sur le bureau.
Prix : 39.90 €
Couleurs disponibles : aluminium brossé
Catégorie : office
Stock : Stock limité
Points forts :
 - Hauteur ajustable
 - Compatible jusqu’à 17 pouces
 - Gestion des câbles intégrée


## CSV File extraction

In [12]:

df_products_csv = normalize_csv_products(df_csv_raw)

In [13]:
df_products_csv

,product_name,price,category,colors,description_short,features,in_stock,raw_file
0,Bureau assis-debout FlexDesk,399.0,office,"[blanc, chêne clair]",Bureau réglable en hauteur pour alterner entre...,"[Réglage électrique, Mémoire 3 positions, Plat...",True,catalogue_multi.csv
1,Lampe de bureau Lumina Pro,59.9,"""electronics / lighting""","[noir, argent]",Lampe LED avec luminosité et température de co...,"[Bras articulé, Lumière chaude/froide, Port US...",True,catalogue_multi.csv
2,Écran 27'' UltraView,279.0,electronics,[noir],"Écran 27 pouces avec bordures fines, idéal pou...","[Résolution 1440p, Mode low, blue light, Pied ...",True,catalogue_multi.csv
3,Chaise de gaming ComfortSeat,199.0,furniture,"[noir, rouge]",Chaise confortable avec support lombaire pour ...,"[Dossier inclinable, Coussin lombaire, Roulett...",True,catalogue_multi.csv
4,Sac à dos UrbanTech,89.0,accessory,"[noir, bleu marine]",Sac à dos pour laptop avec multiples compartim...,"[Compartiment 15 pouces, Poche cachée, Port US...",True,catalogue_multi.csv
5,Casque sans fil FocusLite,79.9,electronics / audio,[gris clair],Casque léger sans fil pensé pour le travail co...,"[Réduction passive de bruit, Micro intégré, Co...",True,catalogue_multi.csv
6,Tapis de souris XL Work&Play,29.9,accessory,[noir],"Grand tapis de souris pour clavier + souris, i...","[Base antidérapante, Surface lisse, Bords renf...",True,catalogue_multi.csv
7,Montre connectée MoveFit,149.0,wearable,"[rose, noir]",Montre connectée pour le suivi d’activité et l...,"[GPS intégré, Suivi du sommeil, Étanche 5ATM]",True,catalogue_multi.csv
8,Chaise de bureau ErgoFlex,249.0,furniture,[noir],Chaise ergonomique avec soutien lombaire régla...,"[Accoudoirs 3D, Dossier mesh respirant, Hauteu...",True,chaise_bureau.csv
9,Clavier mécanique FusionKey 75,119.9,electronics,"[noir, blanc cassé]","Clavier mécanique compact 75 %, switches tacti...","[Switches hot, swap, Rétroéclairage ajustable,...",True,clavier_mecanique.csv


## TXT files extraction using api

In [14]:
df_products_txt = extract_products_from_txt_df(df_txt_raw)

In [15]:
df_products_txt

,product_name,price,category,colors,description_short,features,in_stock,raw_file
0,Montre connectée AX490,129.0,wearable,"[noir, bleu]",Montre élégante pour le quotidien avec capteur...,"[Ultra légère, Compatible iOS et Android, Noti...",False,fiche1_montre.txt
1,Casque audio NovaSound Pro,89.9,electronics / audio,[noir mat],Casque circum-aural à réduction de bruit activ...,"[Réduction de bruit active, Autonomie jusqu'à ...",True,fiche2_casque.txt
2,Sac à dos UrbanTrail 25L,59.0,accessory,"[gris, vert olive]",Sac à dos conçu pour la ville et les petites r...,"[Compartiment ordinateur 15 pouces, Poche secr...",False,fiche3_sac_a_dos.txt
3,T-shirt technique RunDry,24.5,clothing,[bleu marine],"T-shirt respirant pour le running, sèche rapid...","[Matière respirante, Coutures plates anti-irri...",True,fiche4_tee_shirt.txt
4,Lampe de bureau FlexiBright,39.0,electronics / lighting,[blanche],"Une lampe LED avec bras flexible, 3 niveaux de...","[Faible consommation, Température de couleur a...",False,fiche5_lampe.txt
5,Montre connectée WorkLife X2,179.0,wearable,"[noir, argent]",Montre connectée pensée pour les professionnel...,"[Autonomie jusqu’à 7 jours, Paiement NFC, Comp...",True,fiches_multi.txt#part1
6,Lampe de bureau MorningBright,49.9,electronics / lighting,[blanc cassé],"Lampe de bureau LED avec lumière réglable, idé...","[Trois niveaux de luminosité, Mode lumière cha...",True,fiches_multi.txt#part2
7,Support d’ordinateur ErgoLift,39.9,office,[aluminium brossé],"Support surélevé pour laptop, conçu pour améli...","[Hauteur ajustable, Compatible jusqu’à 17 pouc...",True,fiches_multi.txt#part3
8,Casque filaire OfficeTalk,49.0,electronics / audio,[noir],"Casque filaire léger avec micro, parfait pour ...","[Micro avec réduction de bruit, Télécommande s...",True,fiches_multi.txt#part4


## Normalisation and validation

Now that we have the good columns, and that our features are well parsed, let's Join txt and csv data frame

In [16]:
df_all_products = pd.concat(
    [df_products_csv, df_products_txt],
    ignore_index=True
)

df_all_products


,product_name,price,category,colors,description_short,features,in_stock,raw_file
0,Bureau assis-debout FlexDesk,399.0,office,"[blanc, chêne clair]",Bureau réglable en hauteur pour alterner entre...,"[Réglage électrique, Mémoire 3 positions, Plat...",True,catalogue_multi.csv
1,Lampe de bureau Lumina Pro,59.9,"""electronics / lighting""","[noir, argent]",Lampe LED avec luminosité et température de co...,"[Bras articulé, Lumière chaude/froide, Port US...",True,catalogue_multi.csv
2,Écran 27'' UltraView,279.0,electronics,[noir],"Écran 27 pouces avec bordures fines, idéal pou...","[Résolution 1440p, Mode low, blue light, Pied ...",True,catalogue_multi.csv
3,Chaise de gaming ComfortSeat,199.0,furniture,"[noir, rouge]",Chaise confortable avec support lombaire pour ...,"[Dossier inclinable, Coussin lombaire, Roulett...",True,catalogue_multi.csv
4,Sac à dos UrbanTech,89.0,accessory,"[noir, bleu marine]",Sac à dos pour laptop avec multiples compartim...,"[Compartiment 15 pouces, Poche cachée, Port US...",True,catalogue_multi.csv
5,Casque sans fil FocusLite,79.9,electronics / audio,[gris clair],Casque léger sans fil pensé pour le travail co...,"[Réduction passive de bruit, Micro intégré, Co...",True,catalogue_multi.csv
6,Tapis de souris XL Work&Play,29.9,accessory,[noir],"Grand tapis de souris pour clavier + souris, i...","[Base antidérapante, Surface lisse, Bords renf...",True,catalogue_multi.csv
7,Montre connectée MoveFit,149.0,wearable,"[rose, noir]",Montre connectée pour le suivi d’activité et l...,"[GPS intégré, Suivi du sommeil, Étanche 5ATM]",True,catalogue_multi.csv
8,Chaise de bureau ErgoFlex,249.0,furniture,[noir],Chaise ergonomique avec soutien lombaire régla...,"[Accoudoirs 3D, Dossier mesh respirant, Hauteu...",True,chaise_bureau.csv
9,Clavier mécanique FusionKey 75,119.9,electronics,"[noir, blanc cassé]","Clavier mécanique compact 75 %, switches tacti...","[Switches hot, swap, Rétroéclairage ajustable,...",True,clavier_mecanique.csv


In [17]:
df_validated = add_validation_columns(df_all_products)

In [19]:
df_validated

,product_name,price,category,colors,description_short,features,in_stock,raw_file,validation_status,validation_errors
0,Bureau assis-debout FlexDesk,399.0,office,"[blanc, chêne clair]",Bureau réglable en hauteur pour alterner entre...,"[Réglage électrique, Mémoire 3 positions, Plat...",True,catalogue_multi.csv,ok,[]
1,Lampe de bureau Lumina Pro,59.9,"""electronics / lighting""","[noir, argent]",Lampe LED avec luminosité et température de co...,"[Bras articulé, Lumière chaude/froide, Port US...",True,catalogue_multi.csv,error,"[category '""electronics / lighting""' is not al..."
2,Écran 27'' UltraView,279.0,electronics,[noir],"Écran 27 pouces avec bordures fines, idéal pou...","[Résolution 1440p, Mode low, blue light, Pied ...",True,catalogue_multi.csv,ok,[]
3,Chaise de gaming ComfortSeat,199.0,furniture,"[noir, rouge]",Chaise confortable avec support lombaire pour ...,"[Dossier inclinable, Coussin lombaire, Roulett...",True,catalogue_multi.csv,ok,[]
4,Sac à dos UrbanTech,89.0,accessory,"[noir, bleu marine]",Sac à dos pour laptop avec multiples compartim...,"[Compartiment 15 pouces, Poche cachée, Port US...",True,catalogue_multi.csv,ok,[]
5,Casque sans fil FocusLite,79.9,electronics / audio,[gris clair],Casque léger sans fil pensé pour le travail co...,"[Réduction passive de bruit, Micro intégré, Co...",True,catalogue_multi.csv,ok,[]
6,Tapis de souris XL Work&Play,29.9,accessory,[noir],"Grand tapis de souris pour clavier + souris, i...","[Base antidérapante, Surface lisse, Bords renf...",True,catalogue_multi.csv,ok,[]
7,Montre connectée MoveFit,149.0,wearable,"[rose, noir]",Montre connectée pour le suivi d’activité et l...,"[GPS intégré, Suivi du sommeil, Étanche 5ATM]",True,catalogue_multi.csv,ok,[]
8,Chaise de bureau ErgoFlex,249.0,furniture,[noir],Chaise ergonomique avec soutien lombaire régla...,"[Accoudoirs 3D, Dossier mesh respirant, Hauteu...",True,chaise_bureau.csv,ok,[]
9,Clavier mécanique FusionKey 75,119.9,electronics,"[noir, blanc cassé]","Clavier mécanique compact 75 %, switches tacti...","[Switches hot, swap, Rétroéclairage ajustable,...",True,clavier_mecanique.csv,ok,[]


## API use for ChatBot : summarize, translate

In [15]:

catalog_df = df_validated.copy()  
CATALOG_TEXT = build_catalog_for_llm(catalog_df)

In [16]:
context = create_catalog_system_context(CATALOG_TEXT)

In [17]:
dashboard = build_catalog_chat_dashboard(context)
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'6a562aea-7491-4586-9dfd-a3b380ef4f6b': {'version…

## Data export

exporting the data into their file

In [ ]:
json_path, csv_path = export_products(df_validated)